In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# EDA – Análisis de Retención de Clientes (Telecom Churn)
Objetivo: Identificar los factores clave que influyen en la baja de clientes (Churn) y proponer estrategias basadas en datos.

In [ ]:
df = pd.read_csv("../../../data/telecom.csv")
df.dropna()
df.drop_duplicates()
df.head()

In [ ]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors="coerce")
df["MonthlyCharges"] = pd.to_numeric(df["MonthlyCharges"],errors="coerce")
df.info()

In [ ]:
cols_binarias = ["TechSupport","Churn","OnlineSecurity","PaperlessBilling"]

for col in cols_binarias:
    df[col] = df[col].apply(lambda x: 1 if x == "Yes" else 0)

df.head()

### Análisis de Correlación Directa
Identificamos qué variables tienen mayor relación positiva o negativa con la baja del cliente (Churn).

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(numeric_only=True),annot=True,cmap="coolwarm",fmt=".2f")
plt.title("Mapa de correlacion: Que afecta al Churn?")
plt.xticks(rotation=45)
plt.show()

### Hallazgo General: El mapa de calor nos permite identificar que variables actúan como anclas (retienen al cliente) y cuales actuan como motores de fuga (lo empujan a irse).

**Factores de Anclaje** (Correlación Negativa)
- Antigüedad (Tenure) [-0.35]: Es la variable mas fuerte. A mayor tiempo en la empresa, la probabilidad de baja cae drasticamente. Los clientes nuevos son nuestro segmento mas vulnerable.
- Servicios de Valor (TechSupport/OnlineSecurity): Ambos muestran correlaciones negativas. Ofrecer estos servicios "ata" al cliente a la empresa de forma positiva.
- La Paradoja de TotalCharges [-0.20]: Aunque parece un retenedor, su valor negativo se debe a que esta atado al Tenure. Solo los clientes que no se van logran acumular cargos totales altos.

**Motores de Fuga** (Correlación Positiva)
- Cargo Mensual (MonthlyCharges) [+0.19]: El precio es un factor de riesgo directo. Cuando la factura sube, el cliente empieza a buscar alternativas.
- Factura Digital (PaperlessBilling) [+0.19]: Los clientes que reciben su factura por mail o app tienen mas facilidad para gestionar la baja de forma digital, lo que aumenta la volatilidad comparado con quienes usan metodos tradicionales.
- SeniorCitizen [+0.15]: Los adultos mayores en este dataset muestran una tendencia mayor a la baja, posiblemente por sensibilidad a los aumentos de precio.

In [ ]:
df.groupby("Contract")["Churn"].mean().sort_values(ascending=False).plot(kind="bar")
plt.title("Porcentaje de Fuga por Tipo de Contrato")
plt.ylabel("Proporcion de Bajas")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Hallazgo Clave: Aunque el tipo de contrato es una variable categorica y no aparece directamente en el mapa de correlacion numerico, es el principal predictor de baja en el negocio.

- Contrato Mes a Mes (Month-to-month): Presenta la tasa de fuga mas alta. Al no existir un compromiso de permanencia, el cliente se retira ante el primer inconveniente o aumento.

**Conclusión**: La estabilidad financiera de la empresa depende de migrar a los clientes del esquema "mes a mes" hacia contratos anuales mediante incentivos o beneficios exclusivos.

In [ ]:
df.groupby("InternetService")["Churn"].mean().sort_values(ascending=False).plot(kind="bar")
plt.title("Porcentaje de Fuga por Servcio de Intenet")
plt.ylabel('Proporcion de Bajas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Hallazgo Clave: Existe una brecha crítica de lealtad según el tipo de infraestructura tecnológica contratada.
- Fuga en Fibra Óptica: Es el servicio con mayor tasa de abandono, alcanzando aproximadamente un 42% de bajas.
- Estabilidad en DSL: Los clientes con esta tecnología son mucho mas leales, con una tasa de fuga de apenas el 19%.
- Segmento solo Telefonia: Los clientes sin servicio de internet presentan la menor volatilidad de todo el dataset.

Conclusión: La Fibra Optica, aunque es el producto premium, es el principal motor de fuga debido a su alto costo. La estrategia debe centrarse en "blindar" a estos clientes vinculandolos a servicios de soporte tecnico para justificar el precio y mejorar la experiencia.

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x='InternetService', y='Churn', hue='Contract', data=df)
plt.title('Tasa de Fuga por Internet y Contrato')
plt.ylabel('Tasa de Fuga (Media de Churn)')
plt.show()

In [ ]:
# Comprobación de cargos mensuales por tipo de servicio y contrato
plt.figure(figsize=(10,6))
sns.boxplot(x='InternetService', y='MonthlyCharges', hue='Contract', data=df)
plt.title('Distribución de Pagos por Internet y Contrato')
plt.show()

### Hallazgos Clave: El Impacto de la tecnologia, el contrato y los costos en la Fuga
- Relacion Tecnología-Fuga: El gráfico de barras muestra que la Fibra Optica tiene la tasa de abandono mas alta, superando el 50% cuando el contrato es Mes a Mes. En contraste, los contratos de dos años en esta misma tecnología reducen la fuga a niveles mínimos.
- Validacion de Costos: El boxplot confirma que los clientes de Fibra Optica enfrentan los cargos mensuales mas altos, con una mediana que ronda los $80 - $90 USD.
- Contraste con DSL: Mientras que en DSL los pagos son menores (mediana cercana a $50 USD) y la fuga es baja.

**Conclusion**: Los datos demuestran que el problema de retención se concentra en los clientes que pagan más. La combinación de un cargo mensual elevado (>$80) y la libertad de un contrato mensual resulta en que 1 de cada 2 clientes de Fibra Óptica abandone el servicio.

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Churn', y='MonthlyCharges', data=df)
plt.title('Distribución de Cargos Mensuales: cuanto pagan los que se van?')
plt.show()

### Hallazgo Clave: El analisis de distribucion muestra una diferencia clara en el comportamiento de gasto entre clientes leales y los que abandonan.

- Zona de Riesgo: Los clientes que abandonan el servicio suelen tener cargos mensuales concentrados por encima de los $80 USD.
- Zona de Lealtad: Los clientes que permanecen tienen una mediana de gasto menor (aprox. $65 USD).

**Conclusion**: Existe un inicio crítico de precio. Superar los $80 USD mensuales sin añadir servicios de soporte o seguridad incrementa exponencialmente la probabilidad de que el cliente solicite la baja.

In [ ]:
soporte_analisis = df.groupby("TechSupport")["Churn"].mean()
sns.barplot(x=soporte_analisis.index,y=soporte_analisis.values)
plt.title('Impacto del Soporte Técnico en la Retención')
plt.ylabel('Clientes que se fueron')
plt.xticks([0, 1], ['Sin Soporte (0)', 'Con Soporte (1)'])
plt.show()

### Hallazgo Clave: El Soporte Tecnico actua como un fuerte retenedor de clientes.

- Sin Soporte (0): La tasa de fuga supera el 30%, siendo uno de los puntos de mayor vulnerabilidad para la empresa.
- Con Soporte (1): La probabilidad de abandono se reduce a casi la mitad (aprox. 15%).

**Conclusion**: Existe una correlacion negativa entre el servicio de soporte y el Churn. Los clientes que se sienten "acompañados" técnicamente tienen una lealtad significativamente mayor. Fomentar este servicio en el proceso de venta podria reducir drasticamente las bajas.

In [ ]:
seguridad_analisis = df.groupby("OnlineSecurity")["Churn"].mean()
sns.barplot(x=seguridad_analisis.index,y=seguridad_analisis.values)
plt.title('Impacto de la Seguridad en la Retención')
plt.ylabel('Clientes que se fueron')
plt.xticks([0, 1], ['Sin Seguridad (0)', 'Con Seguridad (1)'])
plt.show()

### Hallazgo Clave: La Seguridad Online es un "ancla" que dificulta la salida del cliente.

- Sin Seguridad (0): Al igual que con el soporte, los clientes sin este servicio adicional presentan una tasa de fuga muy elevada.
- Con Seguridad (1): Los clientes que protegen sus dispositivos con la empresa muestran una tasa de desercion mucho mas baja.

**Conclusion**: Los servicios de seguridad no solo generan ingresos extra, sino que aumentan el "costo de salida" percibido por el usuario. Un cliente con sus datos protegidos por la compañia es un cliente mucho mas difícil de perder frente a ofertas de la competencia.

## Conclusion Final:
Los puntos clave
- El peligro de la fibra con contrato mensual: Es el problema mas grande que encontramos. Cuando juntas una factura alta (mas de 80 dolares) con la libertad de irse cuando quieran, la mitad de los clientes se termina yendo.

- El "seguro" de la empresa: Los contratos de 1 o 2 años son los que realmente mantienen a la gente. No importa si el precio es alto, si hay un compromiso firmado, la fuga es casi cero.

- Los servicios que ayudan: El soporte tecnico y la seguridad online funcionan como un "ancla". El cliente que se siente protegido y ayudado por la empresa tiene mucho mas miedo de irse que el que solo tiene internet. Contar con estos servicios reduce la probabilidad de abandono a la mitad (del 30% al 15% aproximadamente)

- El factor tiempo: Los clientes nuevos son los mas dificiles. A medida que pasan los meses y el cliente se acostumbra al servicio (Tenure), las ganas de irse bajan muchisimo.

En resumen: Para que la gente no se vaya, la empresa no tiene que bajar los precios, tiene que lograr que los clientes de fibra se pasen a contratos largos y darles soporte tecnico gratis para que sientan que lo que pagan vale la pena.